In [1]:
import pandas as pd
import importlib
import time
import os
os.chdir("..")

In [2]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from nltk.tokenize import sent_tokenize
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('spacytextblob')

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
data = pd.read_csv("data/reviews_paris_hotels.csv")
data

,listing_name,listing_score,username,review_score,review_title,review
0,Hotel Moderniste,5.0,Alex M,5.0,Quality over the quantity!,l have stayed in many hotels. But Moderniste i...
1,Hotel Moderniste,5.0,Andrea Saade,5.0,Great experience!,We stayed at the Moderniste in December 2021 d...
2,Hotel Moderniste,5.0,halloCairns,5.0,Just renovated hotel in a nice neighbourhood i...,Very helpful at reception. Good service. Clean...
3,Hotel Malte - Astotel,5.0,Grace L,5.0,Fabulous,Myself & my husband cannot recommend this hote...
4,Hotel Malte - Astotel,5.0,TomosParsons,5.0,"Great location, modern and spacious","Great location, being so close to the Louvre a..."
...,...,...,...,...,...,...
208,St Christopher's Inn Canal Paris,4.0,Fernando T,5.0,Eurotrip,Amazing. The staff Pedro was very kind with us...
209,St Christopher's Inn Canal Paris,4.0,Spencer S,5.0,such an amazing trip,Barry is the sexiest bartender alive i love th...
210,St Christopher's Inn Canal Paris,4.0,Timmy B,4.0,Good value and good staff,"Overall a very good hostel experience, staff e..."
211,St Christopher's Inn Canal Paris,4.0,Teigan L,3.0,Pleasant Hostel,This was a pleasant stay. Very economical and ...


In [6]:
data["review"][0]

'l have stayed in many hotels. But Moderniste is really exceeded my expectation. Manager and all staff were very welcoming. Breakfast was excellent. Room was very nice and very comfortable bed.\n\nWe also love the location. On our next visit we will defiantly stay here.\n\nThank you Moderniste.'

In [7]:
from Preprocessors.ReviewPreprocessor import ReviewPreprocessor

In [8]:
import Preprocessors.ReviewPreprocessor

In [9]:
importlib.reload(Preprocessors.ReviewPreprocessor)

<module 'Preprocessors.ReviewPreprocessor' from 'E:\\PychCarmProjects\\s4\\SentiAnalysis\\Preprocessors\\ReviewPreprocessor.py'>

In [10]:
preprocessor = ReviewPreprocessor(data["review"])
t = time.time()
cleaned_reviews = preprocessor.start()
print(f"{time.time() - t} s")
cleaned_reviews

remove_tags done
spelling_correction done
remove_objective_sentences done
26.31265354156494 s


0      l have stayed in many hotels. Manager and all ...
1      We were a family of 4 (2 adults and 2 kids) an...
2      Good service. Clean. Nice room. The hotel is c...
3      Myself & my husband cannot recommend this hote...
4      Great location, being so close to the Louvre a...
                             ...                        
208    Amazing. The staff Pedro was very kind with us...
209    Barry is the sexiest bartender alive i love th...
210    Overall a very good hostel experience, staff e...
211    This was a pleasant stay. Very economical and ...
212    The room was clean and it seemed to be very ne...
Name: review, Length: 213, dtype: object

In [13]:
data["cleaned_review"] = cleaned_reviews

In [14]:
data["cleaned_review"][3]

'Myself & my husband cannot recommend this hotel highly enough. Rooms were beautiful and spacious, breakfast was lovely with plenty of different options to choose from.'

In [15]:
data["review"][3]

'Myself & my husband cannot recommend this hotel highly enough. From the minute we walked in the staff were all so helpful and welcoming. Rooms were beautiful and spacious, breakfast was lovely with plenty of different options to choose from. We will definitely be back.'

In [56]:
def extract_aspects(review):
    aspects = []
    doc = nlp(review)
    for sentence in doc.sents:
        descriptive_term = ''
        target = ''
        for token in doc:
            if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
                target = token.text
            if token.pos_ == 'ADJ':
                prepend = ''
                for child in token.children:
                    if child.pos_ != 'ADV':
                        continue
                    prepend += child.text + ' '
                descriptive_term = prepend + token.text

            if target != '' and descriptive_term != '' and {'aspect': target, 'description': descriptive_term} not in aspects:
                aspects.append({'aspect': target, 'description': descriptive_term})
    return aspects

In [76]:
extract_aspects(data["cleaned_review"][0])

[{'aspect': 'l', 'description': 'many'},
 {'aspect': 'l', 'description': 'very welcoming'},
 {'aspect': 'Breakfast', 'description': 'very welcoming'},
 {'aspect': 'Breakfast', 'description': 'excellent'},
 {'aspect': 'Room', 'description': 'excellent'},
 {'aspect': 'Room', 'description': 'very nice'},
 {'aspect': 'Room', 'description': 'very comfortable'}]

In [77]:
aspects = []
for id_, review in data["cleaned_review"].items():
    for extracted_aspets in extract_aspects(review):
        aspects.append(extracted_aspets)
print(aspects)

[{'aspect': 'l', 'description': 'many'}, {'aspect': 'l', 'description': 'very welcoming'}, {'aspect': 'Breakfast', 'description': 'very welcoming'}, {'aspect': 'Breakfast', 'description': 'excellent'}, {'aspect': 'Room', 'description': 'excellent'}, {'aspect': 'Room', 'description': 'very nice'}, {'aspect': 'Room', 'description': 'very comfortable'}, {'aspect': 'hotel', 'description': 'new'}, {'aspect': 'amenities', 'description': 'new'}, {'aspect': 'staff', 'description': 'new'}, {'aspect': 'staff', 'description': 'very professional'}, {'aspect': 'staff', 'description': 'extremely nice'}, {'aspect': 'staff', 'description': 'careful'}, {'aspect': 'Cleaners', 'description': 'careful'}, {'aspect': 'Cleaners', 'description': 'very efficient'}, {'aspect': 'Cleaners', 'description': 'noisy'}, {'aspect': 'Cleaners', 'description': 'hot'}, {'aspect': 'Cleaners', 'description': 'available'}, {'aspect': 'kids', 'description': 'available'}, {'aspect': 'holidays', 'description': 'available'}, {'a

In [57]:
data["aspects"] = data["cleaned_review"].apply(lambda review : extract_aspects(review))

In [58]:
data[["cleaned_review", "aspects"]]

,cleaned_review,aspects
0,l have stayed in many hotels. Manager and all ...,"[{'aspect': 'l', 'description': 'many'}, {'asp..."
1,We were a family of 4 (2 adults and 2 kids) an...,"[{'aspect': 'hotel', 'description': 'new'}, {'..."
2,Good service. Clean. Nice room. The hotel is c...,"[{'aspect': 'hotel', 'description': 'Nice'}, {..."
3,Myself & my husband cannot recommend this hote...,"[{'aspect': 'Rooms', 'description': 'beautiful..."
4,"Great location, being so close to the Louvre a...","[{'aspect': 'hotel', 'description': 'central'}..."
...,...,...
208,Amazing. The staff Pedro was very kind with us...,"[{'aspect': 'staff', 'description': 'Amazing'}..."
209,Barry is the sexiest bartender alive i love th...,[]
210,"Overall a very good hostel experience, staff e...","[{'aspect': 'staff', 'description': 'very good..."
211,This was a pleasant stay. Very economical and ...,"[{'aspect': 'lockers', 'description': 'better'..."


In [65]:
data["cleaned_review"][1]

'We were a family of 4 (2 adults and 2 kids) and really enjoyed our time there! The hotel is brand new, decorated very nicely and with quality products (the amenities in the bathroom were L’octane). The staff is very professional, extremely nice and careful. Cleaners were very efficient (and not the noisy staff we sometimes find in hotels and during the 10 days we stayed there both rooms were perfectly looked after. I must add the coffee and hot chocolate machine available all day! My kids loved it! I really thank the staff for making our xmas holidays an enjoyable family time.'

In [59]:
data["aspects"][1]

[{'aspect': 'hotel', 'description': 'new'},
 {'aspect': 'amenities', 'description': 'new'},
 {'aspect': 'staff', 'description': 'new'},
 {'aspect': 'staff', 'description': 'very professional'},
 {'aspect': 'staff', 'description': 'extremely nice'},
 {'aspect': 'staff', 'description': 'careful'},
 {'aspect': 'Cleaners', 'description': 'careful'},
 {'aspect': 'Cleaners', 'description': 'very efficient'},
 {'aspect': 'Cleaners', 'description': 'noisy'},
 {'aspect': 'Cleaners', 'description': 'hot'},
 {'aspect': 'Cleaners', 'description': 'available'},
 {'aspect': 'kids', 'description': 'available'},
 {'aspect': 'holidays', 'description': 'available'},
 {'aspect': 'holidays', 'description': 'enjoyable'}]

In [30]:
doc = nlp("We were a family of 4 (2 adults and 2 kids) and really enjoyed our time there! The hotel is brand new, decorated very nicely and with quality products (the amenities in the bathroom were L’octane). The staff is very professional, extremely nice and careful. Cleaners were very efficient (and not the noisy staff we sometimes find in hotels and during the 10 days we stayed there both rooms were perfectly looked after. I must add the coffee and hot chocolate machine available all day! My kids loved it! I really thank the staff for making our xmas holidays an enjoyable family time.")

In [71]:
from spacy import displacy

In [73]:
list(doc.sents)[1]

The hotel is brand new, decorated very nicely and with quality products (the amenities in the bathroom were L’octane).